In [73]:
import os
import numpy as np
from PIL import Image

dataset_path = r"C:\Users\asus\Documents\MANAS\train\train"
input_size = 28 * 28
hidden_size = 64
output_size = 10

class_folders = sorted(os.listdir(dataset_path))
class_mapping = {folder: idx for idx, folder in enumerate(class_folders)}
num_classes = len(class_mapping)

all_image_data = []
all_labels = []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    class_label = class_mapping[folder]

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        image = Image.open(image_path)
        image_data = np.array(image).flatten() / 255.0

        label = class_label  
        all_image_data.append(image_data)
        all_labels.append(label)


all_image_data = np.array(all_image_data)
all_labels = np.array(all_labels)

print("All image data shape:", all_image_data.shape)
print("All labels shape:", all_labels.shape)

m = all_image_data.shape[0]

def init_params():
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X.T) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return (Z > 0).astype(int)

def one_hot(Y, num_classes):
    one_hot_Y = np.zeros((num_classes, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y, output_size)
    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * dZ1.dot(X)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    predictions = np.zeros(A2.shape[1], dtype=int)
    for i in range(A2.shape[1]):
        predictions[i] = np.argmax(A2[:, i])
    return predictions

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(total_images):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration: {i}, Accuracy: {accuracy}")
    return W1, b1, W2, b2

num_images_per_class = 800
total_images = num_images_per_class * num_classes

W1, b1, W2, b2 = gradient_descent(all_image_data[:total_images], all_labels[:total_images], 0.01, 1000)


All image data shape: (8000, 784)
All labels shape: (8000,)
Iteration: 0, Accuracy: 0.0975
Iteration: 10, Accuracy: 0.096625
Iteration: 20, Accuracy: 0.098
Iteration: 30, Accuracy: 0.09875
Iteration: 40, Accuracy: 0.099375
Iteration: 50, Accuracy: 0.100375
Iteration: 60, Accuracy: 0.101125
Iteration: 70, Accuracy: 0.1015
Iteration: 80, Accuracy: 0.101875
Iteration: 90, Accuracy: 0.10175
Iteration: 100, Accuracy: 0.101625
Iteration: 110, Accuracy: 0.1015
Iteration: 120, Accuracy: 0.101375
Iteration: 130, Accuracy: 0.1015
Iteration: 140, Accuracy: 0.1015
Iteration: 150, Accuracy: 0.102
Iteration: 160, Accuracy: 0.10025
Iteration: 170, Accuracy: 0.099625
Iteration: 180, Accuracy: 0.099375
Iteration: 190, Accuracy: 0.099625
Iteration: 200, Accuracy: 0.10025
Iteration: 210, Accuracy: 0.10075
Iteration: 220, Accuracy: 0.102875
Iteration: 230, Accuracy: 0.105875
Iteration: 240, Accuracy: 0.108625
Iteration: 250, Accuracy: 0.109625
Iteration: 260, Accuracy: 0.109375
Iteration: 270, Accuracy: 0

In [76]:
test_dataset_path = r"C:\Users\asus\Documents\MANAS\test\test"
num_test_images_per_class = 200
total_test_images = num_test_images_per_class * num_classes

test_image_data = []
test_labels = []

for folder in os.listdir(test_dataset_path):
    folder_path = os.path.join(test_dataset_path, folder)
    class_label = class_mapping[folder]

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        image = Image.open(image_path)
        image_data = np.array(image).flatten() / 255.0

        label = class_label
        test_image_data.append(image_data)
        test_labels.append(label)

test_image_data = np.array(test_image_data)
test_labels = np.array(test_labels)

test_predictions = forward_prop(W1, b1, W2, b2, test_image_data)[-1]
test_predictions = get_predictions(test_predictions)
test_accuracy = get_accuracy(test_predictions, test_labels)

print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.2815
